<figure>
<img src="../Imagenes/logo-final-ap.png"  width="80" height="80" align="left"/> 
</figure>

# <span style="color:blue"><left>Aprendizaje Profundo</left></span>

# <span style="color:red"><center>Datasets y Dataloaders: CreditCard</center></span>

<figure>
<center>
<img src="../Imagenes/Data_ST.jpg" width="400" height="400" align="center"/>
</center>
</figure>


Fuente: <a href="https://commons.wikimedia.org/wiki/File:Data_ST.jpg">Glenn_Cote_Brent_Spiner.jpg: Grcotederivative work: El Carlos</a>, <a href="http://www.gnu.org/copyleft/fdl.html">GFDL</a>, via Wikimedia Commons<a 

## <span style="color:#4361EE">Coordinador</span>

- Campo Elías Pardo, PhD, cepardot@unal.edu.co

## <span style="color:#4361EE">Profesores</span>

1. Alvaro  Montenegro, PhD, ammontenegrod@unal.edu.co
1. Camilo José Torres Jiménez, Msc, cjtorresj@unal.edu.co
1. Daniel  Montenegro, Msc, dextronomo@gmail.com 

## <span style="color:#4361EE">Asesora Medios y Marketing digital</span>

1. Maria del Pilar Montenegro, pmontenegro88@gmail.com
1. Jessica López Mejía, jelopezme@unal.edu.co
1. Venus Celeste Puertas Gualtero, vpuertasg@unal.edu.co

## <span style="color:#4361EE">Jefe Jurídica</span>

6. Paula Andrea Guzmán, guzmancruz.paula@gmail.com

## <span style="color:#4361EE">Coordinador Jurídico</span>

7. David Fuentes, fuentesd065@gmail.com

## <span style="color:blue">Contenido</span> 

* [Introducción](#Introducción)
* [Importa módulos](#Importa-módulos)
* [Los datos](#Los-datos)
* [Lee los datos](#Lee-los-datos)
* [Crea los datasets](#Crea-los-datasets)
* [Crea los  dataloaders](#Crea-los-datasets)


## <span style="color:blue">Introducción</span> 

En esta lección hacemos ejemplo de clasificación para detectar fraude en el uso de trajeta de crédito. El ejemplo es adaptado de  [fchollet](https://github.com/keras-team/keras-io/blob/master/examples/structured_data/imbalanced_classification.py). Los datos están disponible en [Kaggle Credit Card Fraud Detection](https://www.kaggle.com/mlg-ulb/creditcardfraud/).

El propósito del ejemplo es ilustrar el caso de datos categóricos altamente desbalanceados. Un  problema muy común en la práctica. Usaremos varias métricas para evaluar el modelo.

La técnica se puede extender a la detección de datos anómalos en grades conjuntos de datos.

## <span style="color:blue">Importa módulos</span> 

In [6]:

import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset

import pandas as pd
import numpy as np

torch.__version__

'2.0.1'

## <span style="color:blue">Los datos</span> 

Excepto por la transacción y el monto, no sabemos cuáles son las otras columnas (por razones de privacidad). Lo único que sabemos es que esas columnas que se desconocen ya se han escalado. Son 284.807 registros y 31 variables.

- *El monto de la transacción* es relativamente pequeño. La media de todas las transacciones realizadas es de aproximadamente USD 88.

- *No hay valores faltantes*

- *La mayoría de las transacciones fueron no fraudulentas* (99,82%) del tiempo, mientras que las transacciones fraudulentas ocurren (0.18%) del tiempo en el marco de datos.

- *Transformación PCA*: La descripción de los datos dice que todas las características pasaron por una transformación PCA (técnica de reducción de dimensionalidad) (excepto por tiempo y cantidad).

- *Escalado*: tenga en cuenta que para implementar una transformación de PCA es necesario escalar previamente las variables (features). (En este caso, todas las variables del grupo  V se han escalado o al menos eso es lo que suponemos que hicieron las personas que desarrollan el conjunto de datos).

Los datos fueron tomados de [Kaggle](https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud). Están  organizados en una archivo de tipo csv.

## <span style="color:blue">Lee los datos</span> 

In [4]:
fname = '../Datos/credit_card/creditcard.csv'
training = pd.read_csv(fname)
training

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


## <span style="color:blue">Preprocesamiento</span> 

### <span style="color:#4CC9F0">Construye tensores para entrada y salida</span>

In [10]:
# Separa features y targest
targets = training.pop('Class')
targets = np.array(targets, dtype = 'uint8')
targets.reshape((targets.shape[0],1))
#
features = np.array(training,dtype = 'float32' )
#
print("tamaño de los targets: ",targets.shape)
print("tamaño de los features: ",features.shape)

tamaño de los targets:  (284807,)
tamaño de los features:  (284807, 30)


### <span style="color:#4CC9F0">Prepara datos de entrenamiento y de validación</span> 

In [11]:
#
num_val_samples = int(len(features) * 0.2)
train_features = features[:-num_val_samples]
train_targets = targets[:-num_val_samples]
val_features = features[-num_val_samples:]
val_targets = targets[-num_val_samples:]

print("Number of training samples:", len(train_features))
print("Number of validation samples:", len(val_features))

Number of training samples: 227846
Number of validation samples: 56961


## <span style="color:blue">Crea los datasets</span> 

In [61]:
# Aplique transformaciones a los datos en esta sección
# ...

# Convertir los arrays de NumPy a tensores de PyTorch
train_data_tensor = torch.from_numpy(train_features)
train_labels_tensor = torch.from_numpy(train_targets)

test_data_tensor = torch.from_numpy(val_features)
test_labels_tensor = torch.from_numpy(val_targets)

# Crear los datasets 
train_dataset = TensorDataset(train_data_tensor, train_labels_tensor)
test_dataset = TensorDataset(test_data_tensor, test_labels_tensor)

## <span style="color:blue">Crea los dataloaders</span> 

In [62]:
num_workers = torch.get_num_threads()//2

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, 
                               num_workers= num_workers)

test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [63]:
next(iter(train_dataloader))

[tensor([[ 4.8304e+04,  2.9743e-01, -1.8310e+00,  6.4397e-01,  3.6930e-01,
          -1.2100e+00,  1.0157e+00, -4.0569e-01,  4.0148e-01,  9.6165e-01,
          -5.4695e-01,  1.2819e+00,  1.2037e+00, -5.5167e-01, -1.1553e-01,
          -3.9395e-01, -4.3896e-01,  4.9590e-01, -9.0876e-01, -9.5203e-02,
           6.6482e-01,  1.1416e-01, -3.2136e-01, -2.2335e-01, -1.9223e-01,
          -2.0618e-01,  9.4233e-01, -8.4073e-02,  6.7043e-02,  4.1403e+02],
         [ 4.1157e+04,  1.0982e+00, -4.1688e-01,  3.5006e-01,  4.3139e-01,
          -6.4329e-01, -1.6607e-01, -2.6239e-01,  1.0190e-01,  6.5095e-01,
          -1.2841e-01,  4.5430e-01,  4.4645e-01, -1.2526e+00,  2.2161e-01,
          -8.7906e-01, -3.5725e-02, -1.6739e-01,  1.4724e-01,  7.2309e-01,
          -3.4177e-02, -4.3004e-02, -1.0769e-01, -1.7467e-01,  5.8096e-02,
           4.9031e-01,  6.7138e-01, -5.7307e-02,  1.8665e-03,  6.9800e+01],
         [ 1.3917e+05,  2.0058e+00,  5.0791e-01, -2.3525e+00,  1.4790e+00,
           1.0410e+00, 